In [1]:
#import packages
from mypackage.constants import SORTER, DRIVER_A, DRIVER_B, KK_DICT, SET_DICT, SET_SERIES, F_SERIES, P_SERIES, C_SERIES, B_SERIES, SHOP_NAME
from mypackage.helpers import Helper
import pandas as pd
from datetime import date, timedelta
import os
import dataframe_image as dfi
import logging

import win32com.client
from PIL import ImageGrab

#Setup
pd.set_option('display.max_rows', 1000)

# KSHYK2

In [2]:
def process_input_hyk(df_hyk, pick_up_point_name_hyk,):
    '''
    clean df_hyk
    '''
    #https://stackoverflow.com/questions/23482668/sorting-by-a-custom-list-in-pandas/27255567
    # Create the dictionary that defines the order for sorting
    sorter_index = dict(zip(SORTER, range(len(SORTER))))
    menu_name = f'Menu (Select One)'

    # Generate a rank column that will be used to sort the dataframe numerically
    df_hyk['Point_Rank'] = df_hyk[pick_up_point_name_hyk].map(sorter_index, SORTER)
    
    # Here is the result asked with the lexicographic sort
    # Result may be hard to analyze, so a second sorting is proposed next
    df_hyk.sort_values(['Point_Rank','Menu (Select One)'], 
                ascending = [True, True], inplace = True)
    df_hyk.drop('Point_Rank', 1, inplace = True)
    
    #insert a column of packet numbering
    df_hyk.insert(3, "No.", range(1,len(df_hyk['Name'])+1))  
    
    #create a list of all unique locations.
    unique_points_hyk = df_hyk['HYK Food Pick Up Point'].unique()
    
    #change Menu name containing substring to short form
    #https://stackoverflow.com/questions/39768547/replace-whole-string-if-it-contains-substring-in-pandas
    df_hyk.loc[df_hyk['Menu (Select One)'].str.contains('Spicy Set S'), 'Menu (Select One)'] = 'Spicy S'
    df_hyk.loc[df_hyk['Menu (Select One)'].str.contains('Not Spicy Set N'), 'Menu (Select One)'] = 'Not Spicy N'
    df_hyk.loc[df_hyk['Menu (Select One)'].str.contains('Vegetarian Set V'), 'Menu (Select One)'] = 'Vegetarian V'
    
    # Generate a column for location name conversion
    df_hyk['Location'] = df_hyk['HYK Food Pick Up Point'].map(KK_DICT, SORTER)
    
    # Generate a column for shortform of each set conversion
    df_hyk['Set'] = df_hyk['Menu (Select One)'].map(SET_DICT)
    return df_hyk
    
    
def output_text_hyk(df_hyk, pick_up_point_name_hyk, output_text_path):
    #create a list of all unique locations.
    unique_locations_hyk = df_hyk['Location'].unique()
    #create a list of all unique Menu.
    unique_flavours = df_hyk['Menu (Select One)'].unique()
    #create a list of all unique set shortform
    unique_sets = df_hyk['Set'].unique()
    
    with open(os.path.join(output_text_path, 'HYK Chef - Total Food To Prepare.txt'), 'w') as food_prepare:
        for x in SET_SERIES:
             if x in list(df_hyk['Set']):
                food_prepare.write(f'{x}'.ljust(8) + f'{(df_hyk["Set"] == x).sum()}' + '\n')
        
        if len(set(F_SERIES).intersection(set(list(df_hyk['Set'])))) != 0 :
            food_prepare.write('\n' + 'F_SERIES' + '\n')
        
        for x in F_SERIES:
             if x in list(df_hyk['Set']):
                 food_prepare.write(f'{x}'.ljust(8) + f'{(df_hyk["Set"] == x).sum()}' + '\n')
                    
        if len(set(P_SERIES).intersection(set(list(df_hyk['Set'])))) != 0 :
            food_prepare.write('\n' + 'P_SERIES' + '\n')
        
        for x in P_SERIES:
             if x in list(df_hyk['Set']):
                food_prepare.write(f'{x}'.ljust(8) + f'{(df_hyk["Set"] == x).sum()}' + '\n')
                    
        if len(set(C_SERIES).intersection(set(list(df_hyk['Set'])))) != 0 :
            food_prepare.write('\n' + 'C_SERIES' + '\n')
        
        for x in C_SERIES:
             if x in list(df_hyk['Set']):
                    food_prepare.write(f'{x}'.ljust(8) + f'{(df_hyk["Set"] == x).sum()}' + '\n')
                               
        if len(set(B_SERIES).intersection(set(list(df_hyk['Set'])))) != 0 :
            food_prepare.write('\n' + 'B_SERIES' + '\n')
        
        for x in B_SERIES:
            if x in list(df_hyk['Set']):
                food_prepare.write(f'{x}'.ljust(8) + f'{(df_hyk["Set"] == x).sum()}' + '\n')
        
    
    
    with open(os.path.join(output_text_path, 'HYK Packaging List.txt'), 'w') as packaging_list:
        index = 0

        for location in unique_locations_hyk:
            packaging_list.write(f'{location} = ')
            for x in SET_SERIES:
                if (((df_hyk["Location"] == location)&(df_hyk['Set'] == x )).sum()) != 0:
                    packaging_list.write(f'{x}{((df_hyk["Location"] == location)&(df_hyk["Set"] == x)).sum()}'+"  ")
                    
            for x in F_SERIES:
                if (((df_hyk["Location"] == location)&(df_hyk['Set'] == x )).sum()) != 0:
                    packaging_list.write(f'{x}{((df_hyk["Location"] == location)&(df_hyk["Set"] == x)).sum()}'+"  ")
                    
            for x in P_SERIES:
                if (((df_hyk["Location"] == location)&(df_hyk['Set'] == x )).sum()) != 0:
                    packaging_list.write(f'{x}{((df_hyk["Location"] == location)&(df_hyk["Set"] == x)).sum()}'+"  ")
                    
            for x in C_SERIES:
                if (((df_hyk["Location"] == location)&(df_hyk['Set'] == x )).sum()) != 0:
                    packaging_list.write(f'{x}{((df_hyk["Location"] == location)&(df_hyk["Set"] == x)).sum()}'+"  ")
                    
            for x in B_SERIES:
                if (((df_hyk["Location"] == location)&(df_hyk['Set'] == x )).sum()) != 0:
                    packaging_list.write(f'{x}{((df_hyk["Location"] == location)&(df_hyk["Set"] == x)).sum()}'+"  ")
                    
            packaging_list.write('\n\n')
            
            
        
    
    with open(os.path.join(output_text_path, 'backup HYK Runner - Location List.txt'), 'w') as runner_list:
        for location in unique_locations_hyk:
            runner_list.write(f'{location} = ')
            for x in SET_SERIES:
                if (((df_hyk["Location"] == location)&(df_hyk['Set'] == x )).sum()) != 0:
                    runner_list.write(f'{x}{((df_hyk["Location"] == location)&(df_hyk["Set"] == x)).sum()}'+"  ")
                    
            for x in F_SERIES:
                if (((df_hyk["Location"] == location)&(df_hyk['Set'] == x )).sum()) != 0:
                    runner_list.write(f'{x}{((df_hyk["Location"] == location)&(df_hyk["Set"] == x)).sum()}'+"  ")
                    
            for x in P_SERIES:
                if (((df_hyk["Location"] == location)&(df_hyk['Set'] == x )).sum()) != 0:
                    runner_list.write(f'{x}{((df_hyk["Location"] == location)&(df_hyk["Set"] == x)).sum()}'+"  ")
                    
            for x in C_SERIES:
                if (((df_hyk["Location"] == location)&(df_hyk['Set'] == x )).sum()) != 0:
                    runner_list.write(f'{x}{((df_hyk["Location"] == location)&(df_hyk["Set"] == x)).sum()}'+"  ")
                    
            for x in B_SERIES:
                if (((df_hyk["Location"] == location)&(df_hyk['Set'] == x )).sum()) != 0:
                    runner_list.write(f'{x}{((df_hyk["Location"] == location)&(df_hyk["Set"] == x)).sum()}'+"  ")
                    
            runner_list.write('\n\n')
            
            

def generate_pickup_image_hyk(df_hyk, excel_output_path, image_output_path):
    '''
    Style df_hyk > Export it > Generate image from the excel
    '''
    logging.info(f"Generating pickup image...")
    
    #for excel
    df_style_hyk = df_hyk.style.apply(Helper.highlight_rows_hyk, axis=1)
    writer = pd.ExcelWriter(excel_output_path)
    df_style_hyk.to_excel(writer, sheet_name='Sheet1', index = False)
    
    
    #format for the column width, .set_column(index_start, index_end, width)
    writer.sheets['Sheet1'].set_column(3, 3, 5)
    writer.sheets['Sheet1'].set_column(4, 4, 30)
    writer.sheets['Sheet1'].set_column(5, 5, 40)
    writer.sheets['Sheet1'].set_column(6, 6, 15)
    writer.close()
    
    #for image gen
    df_temp_hyk = df_hyk.iloc[:, 3:7]
    df_style_hyk = df_temp_hyk.style.apply(Helper.highlight_rows_hyk, axis=1)
    #Export as excel
    excel_temp_path = excel_output_path[:-5] + '_for_image_gen.xlsx'
    writer2 = pd.ExcelWriter(excel_temp_path)
    df_style_hyk.to_excel(writer2, sheet_name='Sheet1', index = False)
    writer2.sheets['Sheet1'].set_column(0, 0, 5)
    writer2.sheets['Sheet1'].set_column(1, 1, 30)
    writer2.sheets['Sheet1'].set_column(2, 2, 35)
    writer2.sheets['Sheet1'].set_column(3, 3, 35)
    writer2.close()

    o = win32com.client.Dispatch('Excel.Application')
    wb = o.Workbooks.Open(os.path.join(os.getcwd(), excel_temp_path))
    ws = wb.Worksheets['Sheet1']

    ws.Range(ws.Cells(1,2),ws.Cells(df_temp_hyk.shape[0]+1,df_temp_hyk.shape[1])).CopyPicture(Format=2)

    img = ImageGrab.grabclipboard()
    img.save(image_output_path)
    wb.Close(True)

    logging.info(f"Image generated in {image_output_path}!")
    

def run_hyk(date_str):
    shop_name = SHOP_NAME['KSHYK2'] #or 'HYK' or 'KS'
    #create folders
    Helper.create_folders(shop_name, date_str)
    pick_up_point_name_hyk = "HYK Food Pick Up Point"
    shop_input_path = os.path.join(shop_name, 'input')
    shop_output_path = os.path.join(shop_name, 'output', date_str)

    excel_output_path = os.path.join(shop_output_path, f"HYK {date_str} orders.xlsx")
    image_output_path = os.path.join(shop_output_path, f"HYK {date_str}.jpg")
    
    for file in os.listdir(shop_input_path):
        #if today file exist:
        if ("How Ye Kee" in file) and (date_str in file):
            logging.info(f"Today's {file} found! cleaning the df_hyk...")
            df_hyk = pd.read_csv(os.path.join(shop_input_path, file))
            df_hyk = process_input_hyk(df_hyk, pick_up_point_name_hyk)
            # display(df_hyk)
            output_text_hyk(df_hyk, pick_up_point_name_hyk, shop_output_path) ##print text
            generate_pickup_image_hyk(df_hyk, excel_output_path, image_output_path)
            return df_hyk
        
        logging.info(f"{date_str} file not found in {shop_name}. Exiting the program...")
    return "Break..."

In [3]:
def process_input_ks(df_ks, pick_up_point_name_ks,):
    '''
    clean df_ks
    '''
    #https://stackoverflow.com/questions/23482668/sorting-by-a-custom-list-in-pandas/27255567
    # Create the dictionary that defines the order for sorting
    sorter_index = dict(zip(SORTER, range(len(SORTER))))

    # Generate a rank column that will be used to sort the dataframe numerically
    df_ks['Point_Rank'] = df_ks[pick_up_point_name_ks].map(sorter_index, SORTER)
    
    # Here is the result asked with the lexicographic sort
    # Result may be hard to analyze, so a second sorting is proposed next
    df_ks.sort_values('Point_Rank', ascending = True, inplace = True)
    df_ks.drop('Point_Rank', 1, inplace = True)
    #insert a column of packet numbering
    df_ks.insert(3, "Packet No.", range(1,len(df_ks['Email Address'])+1))  
    #Generate a column for location name conversion
    df_ks['Location'] = df_ks[pick_up_point_name_ks].map(KK_DICT, SORTER)
    return df_ks

def output_text_ks(df_ks, pick_up_point_name_ks, output_text_path):
    # order list
    dishes = df_ks[df_ks.columns[6]].tolist()
    customizes = df_ks[df_ks.columns[7]].fillna(' ').tolist()
    with open(os.path.join(output_text_path, 'KIMSENG Order List.txt'), 'w', encoding="utf-8") as order_list:
        for index, (d, c) in enumerate(zip(dishes, customizes)):
            order_list.write(f'{index+1}. \n')
            for dish in d.split(','):
                dish = dish.strip()
                order_list.write(dish)
                order_list.write('\n')

            for customize in c.split(','):
                customize = customize.strip()
                order_list.write(customize + '\n')

            order_list.write('\n')

    unique_points_ks = df_ks[pick_up_point_name_ks].unique()
    #for loop min() and max() of index of each location into txt file.    
    with open(os.path.join(output_text_path, 'backup KIMSENG Delivery Location.txt'), 'w', encoding="utf-8") as order_location:
        for point in unique_points_ks:
            if df_ks[df_ks[pick_up_point_name_ks]== point]['Packet No.'].min() == df_ks[df_ks[pick_up_point_name_ks]== point]['Packet No.'].max():
                order_location.write(f"{df_ks[df_ks[pick_up_point_name_ks]== point]['Packet No.'].min()}={KK_DICT[point]}"+"\n")
            else:
                order_location.write(f"{df_ks[df_ks[pick_up_point_name_ks]== point]['Packet No.'].min()}-{df_ks[df_ks[pick_up_point_name_ks]== point]['Packet No.'].max()}={KK_DICT[point]}"+"\n")
    
    
    #Packging list
    with open(os.path.join(output_text_path, 'KIMSENG Packaging.txt'), 'w', encoding="utf-8") as order_packaging:
        for point in unique_points_ks:
            if df_ks[df_ks[pick_up_point_name_ks]== point]['Packet No.'].min() == df_ks[df_ks[pick_up_point_name_ks]== point]['Packet No.'].max():
                order_packaging.write(f"{df_ks[df_ks[pick_up_point_name_ks]== point]['Packet No.'].min()} ({df_ks[df_ks[pick_up_point_name_ks]== point]['Packet No.'].max()-df_ks[df_ks[pick_up_point_name_ks]== point]['Packet No.'].min()+1} 包)"+"\n")
            else:
                order_packaging.write(f"{df_ks[df_ks[pick_up_point_name_ks]== point]['Packet No.'].min()}-{df_ks[df_ks[pick_up_point_name_ks]== point]['Packet No.'].max()} ({df_ks[df_ks[pick_up_point_name_ks]== point]['Packet No.'].max()-df_ks[df_ks[pick_up_point_name_ks]== point]['Packet No.'].min()+1} 包)"+"\n")

    
    

def generate_pickup_image_ks(df_ks, excel_output_path, image_output_path):
    '''
    Style df_ks > Export it > Generate image from the excel
    '''
    logging.info(f"Generating pickup image...")

    #for excel
    df_style_ks = df_ks.style.apply(Helper.highlight_rows_ks, axis=1)
    writer = pd.ExcelWriter(excel_output_path)
    df_style_ks.to_excel(writer, sheet_name='Sheet1', index = False)

    #format for the column width, .set_column(index_start, index_end, width)
    writer.sheets['Sheet1'].set_column(3, 3, 5)
    writer.sheets['Sheet1'].set_column(4, 4, 30)
    writer.sheets['Sheet1'].set_column(5, 5, 40)
    writer.sheets['Sheet1'].set_column(6, 6, 45)
    writer.close()
    
    #for image gen
    df_temp_ks = df_ks.iloc[:, 3:6]
    df_style_ks = df_temp_ks.style.apply(Helper.highlight_rows_ks, axis=1)
    #Export as excel
    excel_temp_path = excel_output_path[:-5] + '_for_image_gen.xlsx'
    writer2 = pd.ExcelWriter(excel_temp_path)
    df_style_ks.to_excel(writer2, sheet_name='Sheet1', index = False)
    writer2.sheets['Sheet1'].set_column(0, 0, 10)
    writer2.sheets['Sheet1'].set_column(1, 1, 30)
    writer2.sheets['Sheet1'].set_column(2, 2, 40)
    writer2.close()

    o = win32com.client.Dispatch('Excel.Application')
    wb = o.Workbooks.Open(os.path.join(os.getcwd(), excel_temp_path))
    ws = wb.Worksheets['Sheet1']

    ws.Range(ws.Cells(1,1),ws.Cells(df_temp_ks.shape[0]+1,df_temp_ks.shape[1])).CopyPicture(Format=2)

    img = ImageGrab.grabclipboard()
    img.save(image_output_path)
    wb.Close(True)

    logging.info(f"Image generated in {image_output_path}!")

def run_ks(date_str):
    shop_name = SHOP_NAME['KSHYK2'] #or 'JS'
    #create folders
    Helper.create_folders(shop_name, date_str)
    pick_up_point_name_ks = "KimSeng Food Pick Up Point"
    shop_input_path = os.path.join(shop_name, 'input')
    shop_output_path = os.path.join(shop_name, 'output', date_str)

    excel_output_path = os.path.join(shop_output_path, f"KIMSENG {date_str} orders.xlsx")
    image_output_path = os.path.join(shop_output_path, f"KIMSENG {date_str}.jpg")
    
    for file in os.listdir(shop_input_path):
        #if today file exist:
        if ("Kim Seng" in file) and (date_str in file):
            logging.info(f"Today's {file} found! cleaning the df_ks...")
            df_ks = pd.read_csv(os.path.join(shop_input_path, file))
            df_ks = process_input_ks(df_ks, pick_up_point_name_ks)
            # display(df_ks)
            output_text_ks(df_ks, pick_up_point_name_ks, shop_output_path) ##print text
            generate_pickup_image_ks(df_ks, excel_output_path, image_output_path)
            return df_ks
        
        logging.info(f"{date_str} file not found in {shop_name}. Exiting the program...")
    return "Break..."

In [4]:
def output_text_kshyk(df_ks, df_hyk, pick_up_point_name_ks, pick_up_point_name_hyk, output_text_path):
    
    #create list of locations available today for ks and hyk
    ks_unique_loc = df_ks["KimSeng Food Pick Up Point"].unique()
    hyk_unique_loc = df_hyk[pick_up_point_name_hyk].unique()
    unique_sets = df_hyk['Set'].unique()
    
    #create a combined list of unique locations for both ks and hyk
    kshyk_unique_list = []

    for x in SORTER:
        if (x in hyk_unique_loc) or (x in ks_unique_loc):
            kshyk_unique_list.append(x)
            
            
    #kshyk combined locations output         
    with open(os.path.join(output_text_path, 'KSHYK driver.txt'), 'w', encoding="utf-8") as message_list:    
        for x in kshyk_unique_list:
            message_list.write(f'{KK_DICT[x]}' + " = " )
            if x in ks_unique_loc:
                if df_ks[df_ks[pick_up_point_name_ks]== x]['Packet No.'].min() == df_ks[df_ks[pick_up_point_name_ks]== x]['Packet No.'].max():
                    message_list.write(f"{df_ks[df_ks[pick_up_point_name_ks]== x]['Packet No.'].min()}")
                else:
                    message_list.write(f"{df_ks[df_ks[pick_up_point_name_ks]== x]['Packet No.'].min()}-{df_ks[df_ks[pick_up_point_name_ks]== x]['Packet No.'].max()}") 

            if (x in ks_unique_loc) and (x in hyk_unique_loc):
                message_list.write(" + ")


            if x in hyk_unique_loc:
                for set in SET_SERIES:
                    if (((df_hyk[pick_up_point_name_hyk] == x)&(df_hyk['Set'] == set )).sum()) != 0:
                        message_list.write(f'{set}{((df_hyk[pick_up_point_name_hyk] == x)&(df_hyk["Set"] == set)).sum()}' + ' ')
                
                for set in F_SERIES:
                    if (((df_hyk[pick_up_point_name_hyk] == x)&(df_hyk['Set'] == set )).sum()) != 0:
                        message_list.write(f'{set}{((df_hyk[pick_up_point_name_hyk] == x)&(df_hyk["Set"] == set)).sum()}' + ' ')

                for set in P_SERIES:
                    if (((df_hyk[pick_up_point_name_hyk] == x)&(df_hyk['Set'] == set )).sum()) != 0:
                        message_list.write(f'{set}{((df_hyk[pick_up_point_name_hyk] == x)&(df_hyk["Set"] == set)).sum()}' + ' ')

                for set in C_SERIES:
                    if (((df_hyk[pick_up_point_name_hyk] == x)&(df_hyk['Set'] == set )).sum()) != 0:
                        message_list.write(f'{set}{((df_hyk[pick_up_point_name_hyk] == x)&(df_hyk["Set"] == set)).sum()}' + ' ')
                
                for set in B_SERIES:
                    if (((df_hyk[pick_up_point_name_hyk] == x)&(df_hyk['Set'] == set )).sum()) != 0:
                        message_list.write(f'{set}{((df_hyk[pick_up_point_name_hyk] == x)&(df_hyk["Set"] == set)).sum()}' + ' ')
                

            message_list.write("\n")
    
            
    #part timers Salary
    with open(os.path.join(output_text_path, 'Parttimer Salary.txt'), 'w', encoding="utf-8") as part_timer_salary:    

        duo_basic = 21
        lone_basic = 25
        incentive_per_pack = 0.05
        part_timer_salary.write('duo_basic = ' + f'{duo_basic}, ' + 'lone_basic = ' + f'{lone_basic}, ' + 'incentive_per_pack = ' + f'{incentive_per_pack}')
        part_timer_salary.write('\n\n')


        part_timer_salary.write('DRIVER A' + '\n')

        driver_A_ks_count = 0
        for x in DRIVER_A:
            if (df_ks["KimSeng Food Pick Up Point"] == x).sum() != 0:
                driver_A_ks_count = driver_A_ks_count + (int((df_ks["KimSeng Food Pick Up Point"] == x).sum()))
        part_timer_salary.write('KS packets = ' + f'{driver_A_ks_count}' + '\n') 

        driver_A_hyk_count = 0
        for x in DRIVER_A:
            if (df_hyk["HYK Food Pick Up Point"] == x).sum() != 0:
                driver_A_hyk_count = driver_A_hyk_count + (int((df_hyk["HYK Food Pick Up Point"] == x).sum()))
        part_timer_salary.write('HYK packets = ' + f'{driver_A_hyk_count}' + '\n')   

        driver_A_total_packets = driver_A_ks_count + driver_A_hyk_count

        part_timer_salary.write("TOTAL PACKETS DELIVERED = " + f'{driver_A_total_packets}' + '\n')

        part_timer_salary.write("SALARY = RM" + f'{duo_basic+incentive_per_pack*driver_A_total_packets}' + '\n')

        part_timer_salary.write('\n\n')


        part_timer_salary.write('DRIVER B' + '\n')

        driver_B_ks_count = 0
        for x in DRIVER_B:
            if (df_ks["KimSeng Food Pick Up Point"] == x).sum() != 0:
                driver_B_ks_count = driver_B_ks_count + (int((df_ks["KimSeng Food Pick Up Point"] == x).sum()))
        part_timer_salary.write('KS packets = ' + f'{driver_B_ks_count}' + '\n') 

        driver_B_hyk_count = 0
        for x in DRIVER_B:
            if (df_hyk["HYK Food Pick Up Point"] == x).sum() != 0:
                driver_B_hyk_count = driver_B_hyk_count + (int((df_hyk["HYK Food Pick Up Point"] == x).sum()))
        part_timer_salary.write('HYK packets = ' + f'{driver_B_hyk_count}' + '\n')   

        driver_B_total_packets = driver_B_ks_count + driver_B_hyk_count

        part_timer_salary.write("TOTAL PACKETS DELIVERED = " + f'{driver_B_ks_count+driver_B_hyk_count}' + '\n')

        part_timer_salary.write("SALARY = RM" + f'{duo_basic+incentive_per_pack*driver_B_total_packets}' + '\n')


        total_packets = driver_A_total_packets + driver_B_total_packets
        part_timer_salary.write('\n\n')
        part_timer_salary.write('LONE DRIVER SALARY = ' + f'{lone_basic + total_packets * incentive_per_pack}' + '\n')
    
    

def run_kshyk(date_str):
    shop_name = SHOP_NAME['KSHYK2'] 
    #create folders
    Helper.create_folders(shop_name, date_str)
    pick_up_point_name_ks = "KimSeng Food Pick Up Point"
    pick_up_point_name_hyk = "HYK Food Pick Up Point"
    
    shop_output_path = os.path.join(shop_name, 'output', date_str)
    
    output_text_kshyk(df_ks, df_hyk, pick_up_point_name_ks, pick_up_point_name_hyk, shop_output_path) ##print text

In [5]:
today_str =(date.today() + timedelta(days=1)).strftime('%d_%m_%Y')
# today_str = date.today().strftime('26_12_2022')
print(f"running for hyk {today_str}:")
df_hyk = run_hyk(today_str)
print(f"running for ks {today_str}:")
df_ks = run_ks(today_str)
print(f"running for kshyk {today_str}:")
run_kshyk(today_str)

running for hyk 08_01_2023:


C:\Users\PC\AppData\Local\Temp\ipykernel_22980\3934242798.py:17: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_hyk.drop('Point_Rank', 1, inplace = True)


running for ks 08_01_2023:


C:\Users\PC\AppData\Local\Temp\ipykernel_22980\1702467138.py:15: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_ks.drop('Point_Rank', 1, inplace = True)


running for kshyk 08_01_2023:


In [6]:
#TEST for quantity calculation for account
df_acc = pd.read_excel('account_template.xlsx', sheet_name='Sheet1')
df_acc['quantity'] = df_acc.apply(lambda row: list(df_hyk['Set']).count(row.Code) , axis=1)

shop_name = SHOP_NAME['KSHYK2']
shop_output_path = os.path.join(shop_name, 'output', today_str)
excel_output_path = os.path.join(shop_output_path, f"HYK {today_str} QUANTITY.xlsx")
df_acc.to_excel(excel_output_path)